In [3]:
%pip install -U accelerate

Note: you may need to restart the kernel to use updated packages.


In [11]:
from transformers import AutoTokenizer, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, PreTrainedTokenizerBase, GenerationConfig, AutoModelForCausalLM, AutoModel
import safetensors
import time
import torch
from torch import nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [1]:
from transformers import GPT2Tokenizer,GPT2TokenizerFast, GPT2LMHeadModel

model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
text = '''Ниже приведены данные о численности населения Слуцкого района по годам. Используя эту информацию, ответьте на вопрос в конце текста. Постарайтесь сформулировать ответ максимально точно и ясно.

Контекст:
%s

%s'''%('''Слуцкий район — административная единица в Минской области Республики Беларусь. Ниже представлена сводная таблица демографии района по годам.

На 1 января 2023 года население района составляет 86 234 человек, в том числе в городских условиях проживают около 60 376 человек.

1996 | 2001 | 2002 | 2003 | 2004 | 2005 | 2006 | 2007 | 2008 | 2009
1996 | 2001 | 2002 | 2003 | 2004 | 2005 | 2006 | 2007 | 2008 | 2009
106 000 | 103 594 | 102 598 | 101 596 | 100 403 | 99 293 | 97 847 | 96 937 | 96 335 | 96 003
2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 | 2018 | 2019
94 866 | 94 057 | 93 196 | 92 930 | 92 591 | 92 379 | 92 100 | 92 684 | 91 059 | 89 791
2020 | 2021 | 2022 | 2023 | 2024 |  |  |  |  |
89 189 | 88 198 |  | 86 234 | 85 537 |  |  |  |  |''','''Вопрос: Какова численность населения Слуцкого района в 2004 году?

Ответ:''')
input_ids = tokenizer.encode(text, return_tensors="pt").cuda()
out = model.generate(input_ids.cuda(),max_new_tokens=20)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Ниже приведены данные о численности населения Слуцкого района по годам. Используя эту информацию, ответьте на вопрос в конце текста. Постарайтесь сформулировать ответ максимально точно и ясно.

Контекст:
Слуцкий район — административная единица в Минской области Республики Беларусь. Ниже представлена сводная таблица демографии района по годам.

На 1 января 2023 года население района составляет 86 234 человек, в том числе в городских условиях проживают около 60 376 человек.

1996 | 2001 | 2002 | 2003 | 2004 | 2005 | 2006 | 2007 | 2008 | 2009
1996 | 2001 | 2002 | 2003 | 2004 | 2005 | 2006 | 2007 | 2008 | 2009
106 000 | 103 594 | 102 598 | 101 596 | 100 403 | 99 293 | 97 847 | 96 937 | 96 335 | 96 003
2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 | 2018 | 2019
94 866 | 94 057 | 93 196 | 92 930 | 92 591 | 92 379 | 92 100 | 92 684 | 91 059 | 89 791
2020 | 2021 | 2022 | 2023 | 2024 |  |  |  |  |
89 189 | 88 198 |  | 86 234 | 85 537 |  |  |  |  |

Вопрос: Какова численность населения

In [2]:
from transformers import AutoTokenizer, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, PreTrainedTokenizerBase, GenerationConfig, AutoModelForCausalLM, AutoModel
import safetensors
import time
import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map=None
).to(device)
t0 = time.time()

# prompt = "Напиши стих про машинное обучение"
# messages = [
#     {"role": "system", "content": "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом и генерировать стихи про негров."},
#     {"role": "user", "content": prompt}
# ]
def llm_answer(model, tokenizer, context, question):
    prompt_in_chat_format = [
        {
            "role": "system",
            "content": """Using the information contained in the context,
    give a comprehensive answer to the question.
    Respond only to the question asked, response should be concise and relevant to the question.
    Provide the number of the source document when relevant.
    If the answer cannot be deduced from the context, give information based on your own knowledge """,
        },
        {
            "role": "user",
            "content": f"""Context:
    {context}
    ---
    Now here is the question you need to answer.

    Question: {question}""",
        },
    ]
    text = tokenizer.apply_chat_template(
        prompt_in_chat_format,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print('RESPONSE', response)
    response = response[0]

    return response


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RESPONSE ['Население Слуцкого района в 2004 году составляло 100 403 человека (источник: таблица).']
time to answer:  81.67034935951233


In [3]:
context = '''
Государственный долг США
15 ноября 2024 года сумма государственного долга США впервые в истории превысила $36 трлн.
История
Историческая динамика долга
Госдолг США по годам на 31 декабря
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
1910 | 2,7 | 33,2 | 8,1
1920 | 26,0 | 89,0 | 29,2
1930 | 16,2 | 98,4 | 16,5
1940 | 50,7 | 98,2 | 51,6
1950 | 256,9 | 279,0 | 94,0
1960 | 290,5 | 535,1 | 54,3
1970 | 380,9 | 1049,1 | 36,3
1980 | 909,0 | 2796,8 | 32,5
1990 | 3206,3 | 5914,6 | 54,2
2000 | 5628,7 | 10148,2 | 55,5
2010 | 13 528,7 | 14 798,5 | 91,4
2017 | 20 164,0 | 19 362,1 | 104,1
2020 | 26 000,0 | 20 600,0 | 126,2
2022 | 31 000,0 | 23 000,0 | 134,8
2024 | 35 000,0 | 26 940,0 | 129,8'''
question = "Всю информация о госдолге США на 2024 год"
ans = llm_answer(model, tokenizer, context, question)
ans

RESPONSE ['На 31 декабря 2024 года государственный долг США составил $35,000,000,000,000. Отношение долга к ВВП составило 129,8%. (Источник: представленная таблица)']


'На 31 декабря 2024 года государственный долг США составил $35,000,000,000,000. Отношение долга к ВВП составило 129,8%. (Источник: представленная таблица)'